# Getting all arrivals to all stops of a given line in a given day

## Install dependencies

This notebook requires two dependencies which can be installed with the following command `pip install pandas open-bus-stride-client`.

You can also launch it online at [this URL](https://mybinder.org/v2/gh/hasadna/open-bus-stride-client/HEAD?labpath=notebooks%2Fgetting%20all%20arrivals%20to%20all%20stops%20of%20a%20given%20line%20in%20a%20given%20day.ipynb), when launching online the dependencies are already installed.

## Import dependencies

In [1]:
import datetime

from ipywidgets import DatePicker
from IPython.display import display

# we use pandas to visualize the results we get
import pandas as pd

# The stride client library, used to make the calls to the stride api
import stride

## Pick a date

First, we pick the day we want to analyze, GTFS data queries should be limited to a specific date

You can use the date picker to choose a date which will be used in the next code blocks

In [2]:
date_widget = DatePicker(description='Date:', value=datetime.date(2022,1,23))
display(date_widget)

DatePicker(value=datetime.date(2022, 1, 23), description='Date:')

## Get list of GTFS Routes

For this example we get routes based on route_short_name which is usually the route number in the GTFS data

We look for all routes on the given day matchin route_short_name `360`

In [3]:
gtfs_routes = stride.get('/gtfs_routes/list', {
    'date_from': date_widget.value, 'date_to': date_widget.value,
    'route_short_name': 360,
}, pre_requests_callback='print')
pd.DataFrame(gtfs_routes)

https://open-bus-stride-api.hasadna.org.il/gtfs_routes/list?date_from=2022-01-23&date_to=2022-01-23&route_short_name=360


,id,date,line_ref,operator_ref,route_short_name,route_long_name,route_mkt,route_direction,route_alternative,agency_name,route_type
0,47952,2022-01-23,28774,4,360,היכל המשפט/אבא אבן-ירושלים<->סובת התאנה-אפרת-1#,14360,1,#,אגד תעבורה,3
1,47953,2022-01-23,28775,4,360,סובת התאנה-אפרת<->בנייני האומה/שדרות שז''ר-ירו...,14360,2,#,אגד תעבורה,3
2,48116,2022-01-23,29729,4,360,סובת התאנה-אפרת<->בנייני האומה/שדרות שז''ר-ירו...,14360,2,4,אגד תעבורה,3


## Get the line_refs for these routes

The SIRI data uses line_ref to identify routes, so we need to get the line_refs from above data to be used in requests to get the SIRI data

In [4]:
line_refs = [gtfs_route['line_ref'] for gtfs_route in gtfs_routes]
line_refs

[28774, 28775, 29729]

## Get list of SIRI routes

To be able to query the related SIRI data we need to get the SIRI route identifiers

In [5]:
siri_routes = stride.get('/siri_routes/list', {
    'line_refs': ','.join(map(str,line_refs))
}, pre_requests_callback='print')
siri_route_ids = [siri_route['id'] for siri_route in siri_routes]
pd.DataFrame(siri_routes)

https://open-bus-stride-api.hasadna.org.il/siri_routes/list?line_refs=28774%2C28775%2C29729


,id,line_ref,operator_ref
0,2149,28775,4
1,2152,28774,4
2,3624,29729,4


In [6]:
siri_route_ids

[2149, 2152, 3624]

## Get list of SIRI rides

Based on this list of SIRI route identifiers we can get the list of SIRI rides which occured on these routes

We use the scheduled_start_time field which is populated from the SIRI data to limit the rides to the specific date we want to check

This data contains the duration_minutes field which is the duration of the ride from the SIRI data

In [7]:
siri_rides = stride.get('/siri_rides/list', {
    'scheduled_start_time_from': datetime.datetime.combine(date_widget.value, datetime.time(), datetime.timezone.utc),
    'scheduled_start_time_to': datetime.datetime.combine(date_widget.value, datetime.time(23,59), datetime.timezone.utc),
    'siri_route_ids': ','.join(map(str,siri_route_ids)),
    'order_by': 'scheduled_start_time asc'
}, pre_requests_callback='print')
pd.DataFrame(siri_rides)

https://open-bus-stride-api.hasadna.org.il/siri_rides/list?scheduled_start_time_from=2022-01-23T00%3A00%3A00.000000%2B0000&scheduled_start_time_to=2022-01-23T23%3A59%3A00.000000%2B0000&siri_route_ids=2149%2C2152%2C3624&order_by=scheduled_start_time+asc


,id,siri_route_id,journey_ref,scheduled_start_time,vehicle_ref,updated_first_last_vehicle_locations,first_vehicle_location_id,last_vehicle_location_id,updated_duration_minutes,duration_minutes
0,6382847,2149,2022-01-23-57859359,2022-01-23T03:30:00+00:00,7667469,2022-01-23 05:00:19.697697+00:00,338023517,338067615,2022-01-23 10:00:18.128393+00:00,26
1,6385190,2152,2022-01-23-57859219,2022-01-23T04:00:00+00:00,7668469,2022-01-23 05:00:26.469328+00:00,338088396,338177583,2022-01-23 11:00:17.510985+00:00,28
2,6384972,2149,2022-01-23-57859360,2022-01-23T04:00:00+00:00,7605769,2022-01-23 05:00:25.768295+00:00,338080402,338173691,2022-01-23 11:00:16.916187+00:00,30
3,6387369,2149,2022-01-23-57859361,2022-01-23T04:30:00+00:00,7666869,2022-01-23 06:00:29.084018+00:00,338158485,338282361,2022-01-23 11:00:28.005670+00:00,30
4,6391177,2149,2022-01-23-57859362,2022-01-23T05:00:00+00:00,8212085,2022-01-23 06:00:46.333548+00:00,338311065,338517204,2022-01-23 12:00:29.565259+00:00,39
5,6389899,2152,2022-01-23-57859220,2022-01-23T05:00:00+00:00,79180201,2022-01-23 06:00:37.332745+00:00,338259342,338451889,2022-01-23 12:00:26.949197+00:00,39
6,6392526,2149,2022-01-23-57859363,2022-01-23T05:20:00+00:00,7669769,2022-01-23 06:00:51.862676+00:00,338371142,338565248,2022-01-23 12:00:31.744367+00:00,36
7,6394840,2149,2022-01-23-57859364,2022-01-23T05:40:00+00:00,7666469,2022-01-23 07:00:59.912208+00:00,338484508,338732232,2022-01-23 13:00:38.369494+00:00,46
8,6396949,2149,2022-01-23-57859365,2022-01-23T06:00:00+00:00,79180201,2022-01-23 07:01:11.617046+00:00,338582020,338856756,2022-01-23 13:00:54.215476+00:00,51
9,6396663,2152,2022-01-23-57859221,2022-01-23T06:00:00+00:00,8212085,2022-01-23 07:01:10.352078+00:00,338576404,338819606,2022-01-23 13:00:52.922479+00:00,46


## Choose a ride

There are usually a lot of rides for a given route, let's choose one of the rides to investigave

In [8]:
siri_ride_id = 6402458

## Get the ride-stops for this ride

Siri ride-stops contain all the list of stops on this ride, the vehicle locations are then related to this ride stop

In [9]:
siri_ride_stops = stride.get('/siri_ride_stops/list', {
    'siri_ride_ids': str(siri_ride_id),
    'order_by': 'order asc'
}, pre_requests_callback='print')
pd.DataFrame(siri_ride_stops)

https://open-bus-stride-api.hasadna.org.il/siri_ride_stops/list?siri_ride_ids=6402458&order_by=order+asc


,id,siri_stop_id,siri_ride_id,order
0,165965976,3375,6402458,1
1,165981643,1592,6402458,2
2,165999999,1600,6402458,3
3,166020333,1172,6402458,4
4,166040442,2984,6402458,5
5,166055311,15955,6402458,7
6,166060155,2519,6402458,8
7,166062550,16305,6402458,9
8,166069856,3227,6402458,10
9,166072174,3228,6402458,11


## Find all vehicle locations along this ride

Now we can get the list of all vehicle locations along these ride-stops

Each vehicle location is related to the siri_ride_stop_id which is closest to this location (according to the MOT SIRI data)

In [13]:
pd.DataFrame(stride.get('/siri_vehicle_locations/list', {
    'siri_ride_stop_ids': ','.join([str(siri_ride_stop['id']) for siri_ride_stop in siri_ride_stops]),
    'order_by': 'recorded_at_time asc'
}, pre_requests_callback='print'))

https://open-bus-stride-api.hasadna.org.il/siri_vehicle_locations/list?siri_ride_stop_ids=165965976%2C165981643%2C165999999%2C166020333%2C166040442%2C166055311%2C166060155%2C166062550%2C166069856%2C166072174%2C166074404&order_by=recorded_at_time+asc


,id,siri_snapshot_id,siri_ride_stop_id,recorded_at_time,lon,lat,bearing,velocity,distance_from_journey_start
0,338897987,215956,165965976,2022-01-23T06:55:00+00:00,35.206916,31.785791,168,4,0
1,338903336,215957,165965976,2022-01-23T06:55:00+00:00,35.206916,31.785791,168,4,0
2,338908635,215958,165965976,2022-01-23T06:55:00+00:00,35.206916,31.785791,168,4,0
3,338913913,215959,165965976,2022-01-23T06:55:00+00:00,35.206916,31.785791,168,4,0
4,338919242,215960,165965976,2022-01-23T07:00:02+00:00,35.205318,31.786596,282,0,0
5,338924585,215961,165965976,2022-01-23T07:01:15+00:00,35.204048,31.786872,282,0,172
6,338929863,215962,165981643,2022-01-23T07:02:06+00:00,35.203979,31.787174,282,0,216
7,338935061,215963,165981643,2022-01-23T07:03:15+00:00,35.204170,31.787933,6,0,344
8,338940181,215964,165981643,2022-01-23T07:04:29+00:00,35.203632,31.787902,34,0,390
9,338945342,215965,165981643,2022-01-23T07:05:14+00:00,35.202060,31.787563,250,0,555
